# SIPR模型

现在好像提到SIR或SIPR模型求参数的开源代码不多，但无论SIR也好还是SIPR等模型，其预测都对初始化参数高度敏感，也就是蝴蝶效应，很小的参数变化在后面模型演化后会引起较大的结果差异，因而根据最新的数据求取或调整模型的参数是很重要的事，这里还涉及到更新R0这个非常重要的指标。同时，求得较优参数比较消耗算力，所以这里先将之前开发的SIPR模型R代码（2019年12月21日根据下面链接论文先用R开发了这个SIPR模型）转为Julia代码，代码没有优化，后面可能会有其它的工作到时相关优化等一并再开源到这里，总之，希望自己一点点微薄的工作能对其他人在这场战役中有所帮助🙏。

这里相关初始化参数同R中代码一样引用了同一论文:

[《Modeling Super-spreading Events for Infectious Diseases: Case Study SARS》](https://arxiv.org/pdf/1007.0908.pdf)


另外，在复杂网络，还有一个相关的模型较为常用，就是SIS模型，感兴趣同学可以参考我根据斯坦福大学Matthew O. Jackson教授的Coursera课程[《Social and Economic Networks: Models and Analysis》](https://github.com/Silk-Road/Social-and-Economic-Networks-Models-and-Analysis)而整理的学习笔记：

- [5. SIS Model](https://github.com/Silk-Road/Social-and-Economic-Networks-Models-and-Analysis/blob/master/5.%20Diffusion%20on%20Networks/5.%20SIS%20Model.ipynb)

- [6.Solving the SIS Model](https://github.com/Silk-Road/Social-and-Economic-Networks-Models-and-Analysis/blob/master/5.%20Diffusion%20on%20Networks/6.%20Solving%20the%20SIS%20Model.ipynb)

- [7 . Solving the SIS Model - Ordering](https://github.com/Silk-Road/Social-and-Economic-Networks-Models-and-Analysis/blob/master/5.%20Diffusion%20on%20Networks/7%20.%20%20Solving%20the%20SIS%20Model%20-%20Ordering.ipynb)

至于SIS模型与SIR模型的简单对比差异可以参考：

https://www.researchgate.net/post/SIR_or_SIS_model_which_one_is_more_accurate_for_explaining_the_spread_of_computer_virus_in_a_network

较为详细的推荐参考这篇论文：

[Modeling super-spreading events for SARS》》](https://pdfs.semanticscholar.org/a239/12ed014f492bb54769ce0497c5f0b84499e7.pdf)



In [2]:
using Distances, Distributions, DataFrames

start_time = 0
end_time = 180
Δt = 1
n = Int(ceil(end_time - start_time) / Δt)
b = 0.6489
ν₁ = 0.0836
ν₂ = 0.0794
β = 0.2586

function de_s(β, s, i, p)
    return -β * (i + p) * s
end

function de_i(b, β, i, p, s, ν₁)
    return b * β * (i + p) * s - ν₁ * i
end

function de_p(b, β, i, p, s, ν₂)
    return (1 - b) * β * (i + p) * s - ν₂ * p
end

function de_r(ν₁, ν₂, i, p)
    return ν₁ * i + ν₂ * p
end

function sipr(β, b, ν₁, ν₂, n, i0, Δt=1)
    """
    SIPR model
    """
    S = repeat([0.0], n)
    I = repeat([0.0], n)
    P = repeat([0.0], n)
    R = repeat([0.0], n)
    total = repeat([0.0], n)

    S[1] = 1400000000.0
    I[1] = i0
    P[1] = 0.0
    R[1] = 0.0

    N = S[1] + I[1] + P[1] + R[1]
    β = β/N
    total[1] = N

    for i in 2:n
        S[i] = S[i-1] + de_s(β, S[i-1], I[i-1], P[i-1]) * Δt
        I[i] = I[i-1] + de_i(b, β, I[i-1], P[i-1], S[i-1], ν₁) * Δt
        P[i] = P[i-1] + de_p(b, β, I[i-1], P[i-1], S[i-1], ν₂) * Δt
        R[i] = R[i-1] + de_r(ν₁, ν₂, I[i-1], P[i-1]) * Δt
        total[i] = S[i] + I[i] + P[i] + R[i]
    end

    df = DataFrame(S = S, I = I, P = P, R = R, N = total)
    return df
end

df_sipr = sipr(β, b, ν₁, ν₂, n, 1)
df_sipr[60:70,:]

,S,I,P,R,N
,Float64,Float64,Float64,Float64,Float64
1,1.39998e9,9372.67,5154.5,6758.21,1.4e9
2,1.39997e9,11026.8,6064.2,7951.03,1.4e9
3,1.39997e9,12972.9,7134.45,9354.37,1.4e9
4,1.39997e9,15262.4,8393.57,11005.4,1.4e9
5,1.39996e9,17956.0,9874.9,12947.8,1.4e9
6,1.39995e9,21124.9,11617.6,15233.0,1.4e9
7,1.39994e9,24853.1,13667.9,17921.4,1.4e9
8,1.39993e9,29239.1,16080.1,21084.4,1.4e9
9,1.39992e9,34399.2,18917.8,24805.5,1.4e9


# 计算SIPR模型参数

参数β、b、ν₁、ν₂、i0的选取对模型的精度有明显的影响，通过之前模型的预测数据来模拟真实的观测数据`observed_cases`，大家可以使用真实的历史数据作为`observed_cases`，只要将`time_range`的值设为`observed_cases`的长度即可，以此数据来求取较优的模型参数β、b、ν₁、ν₂、i0，值得注意的是i0较优值可能大于1，从技术上来说第一个感染者很可能在2019年12月1日前就已经出现。

In [4]:
number_samples = 10000000
β_prior = .^(10,rand(Uniform(-1,0), number_samples)) 
b_prior = rand(Uniform(0,1), number_samples)
ν₁_prior = rand(Uniform(0,0.1), number_samples) 
ν₂_prior = rand(Uniform(0,0.1), number_samples)
i0_prior = rand(Poisson(1), number_samples).+1 

N = 1400000000 # 人口总数
time_range = 100 # 100天传染扩散, time_range == length(observed_cases)
threshold = 20000 # 用以存储后验的阀值

β_posterior = Float64[]
b_posterior = Float64[]
ν₁_posterior = Float64[]
ν₂_posterior = Float64[]
i0_posterior = Int64[]
distance_posterior = Float64[]

observed_cases = df_sipr[:I][1:100] # 基于之前论文参数模拟的数据作为观测数据


for i in 1:number_samples
    simulated_timeseries = sipr(β_prior[i], b_prior[i], ν₁_prior[i],
                               ν₂_prior[i], time_range, i0_prior[i])
    distance = evaluate(Euclidean(), observed_cases, simulated_timeseries[2]) 
    if distance <= threshold
        push!(β_posterior, β_prior[i])
        push!(b_posterior, b_prior[i])
        push!(ν₁_posterior, ν₁_prior[i])
        push!(ν₂_posterior, ν₂_prior[i])
        push!(i0_posterior, i0_prior[i])
        push!(distance_posterior, distance)
    end
end

posteriors = DataFrame(β = β_posterior,
                       ν₁ = ν₁_posterior,
                       ν₂ = ν₂_posterior,
                       b = b_posterior,
                       i0 = i0_posterior)


beta = mean(posteriors[:β])
nu_1 = mean(posteriors[:ν₁])
nu_2 = mean(posteriors[:ν₂])
b = mean(posteriors[:b])
i0 = mean(posteriors[:i0])

# SIPR R0计算详见论文：https://pdfs.semanticscholar.org/a239/12ed014f492bb54769ce0497c5f0b84499e7.pdf
R0 = (1 - b) * β / ν₂ + b * β / ν₁

simulated_timeseries = sipr(β, b, ν₁, ν₂, time_range, i0)
simulated_timeseries[60:70,:]

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[4]:18
└ @ Core In[4]:18
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[4]:25
└ @ Core ./In[4]:25
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[4]:41
└ @ Core In[4]:41
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[4]:44
└ @ Core In[4]:44
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[4]:45
└ @ Core In[4]:45
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[4]:46
└ @ Core In[4]:46
┌ Warning: `getindex

,S,I,P,R,N
,Float64,Float64,Float64,Float64,Float64
1,1.39998e9,10370.8,6425.71,7797.73,1.4e9
2,1.39997e9,12202.4,7560.51,9174.93,1.4e9
3,1.39997e9,14357.3,8895.71,10795.4,1.4e9
4,1.39996e9,16892.8,10466.7,12701.9,1.4e9
5,1.39995e9,19876.1,12315.1,14945.2,1.4e9
6,1.39994e9,23386.1,14489.9,17584.7,1.4e9
7,1.39993e9,27516.0,17048.8,20690.3,1.4e9
8,1.39992e9,32375.2,20059.5,24344.3,1.4e9
9,1.39991e9,38092.5,23601.9,28643.6,1.4e9
